In [35]:
from bs4 import BeautifulSoup as bs

API_BASE_URL = 'https://immovlan.be/'
HOUSES_ENDPOINT = '/en/real-estate?transactiontypes=for-sale,in-public-sale&noindex=1'

soup = bs(API_BASE_URL.join(HOUSES_ENDPOINT), 'html.parser')
for e in soup.find_all('article'):
        print(e)

for i in soup.find_all('strong', attrs={'class':'list-item-price'}):
        print (i)


In [40]:
import csv

with open('./src/cities2.csv', newline='') as csvfile:

    csv_reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row, col in enumerate(csv_reader):
        if row == 1:
            print(col[0])

1000


In [ ]:
"""from selenium import webdriver
#from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.edge.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC"""

from urllib.parse import urljoin

from bs4 import BeautifulSoup as bs

import requests
import csv
import re
import time
import json


session = requests.Session()
base_url = 'https://immovlan.be/'

with open('./src/cities2.csv', newline='') as csvfile:

    csv_reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row, col in enumerate(csv_reader):
        if row == 1:
            zip_code = f'{col[0]}-{col[1]}'

page = 1

#zip_code = '1000-Brussels'
# select only houses, appartments
appartment_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=apartmen&towns={zip_code}&page={page}noindex=1'
house_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&towns={zip_code}&noindex=1'

"""
Selenium Stealth Mode modifies browser behavior to mimic real user interactions, helping bypass anti-bot detection systems during web scraping.

options = webdriver.ChromeOptions()
options.headless = True
# to make the browser appear more like a regular user-controlled browser and less like an automated bot
options.add_experimental_option("excludeSwitches", ["enable-automation"])
#options.add_argument("--headless=new")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)
response = driver.get(urljoin(base_url, prop_endpoint))


wait = WebDriverWait(driver, 20)
cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@id='didomi-notice-agree-button']")))
cookie_button.click()
"""

#url = driver.page_source

headers =  {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) '
            'Gecko/20100101 Firefox/115.0'
        ),
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }



req = session.get(urljoin(base_url, appartment_endpoint), headers=headers)
time.sleep(5)
print(req.status_code)
# for specific elements: driver.find_elements_by_xpath(xpath_expression)
soup = bs(req.content, 'lxml')

# Get total listed properties for search
texts = soup.find_all('div', {'class':'col-12 mb-2'})
for text in texts:
    text_listed_properties = text.get_text()

# Get numbers of listed properties
# This pattern matches the beginning of the string ^, skips any non-digit characters \D*, and then captures the first sequence of digits (\d+).
parameter = r'^\D*(\d+)'
total_listed_properties = int(re.findall(parameter, text_listed_properties)[0])
print(total_listed_properties)

# Divide by 20 to assess number of pages
numb_of_pages = int(total_listed_properties/20)
if numb_of_pages%20 > 0:
    numb_of_pages += 1
print(numb_of_pages)

link_list = []
full_link = []

for page in range(numb_of_pages):
    
    appartment_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=apartmen&towns={zip_code}&page={page}noindex=1'
    house_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&towns={zip_code}&noindex=1'

    req = session.get(urljoin(base_url, appartment_endpoint), headers=headers)
    print(req.status_code)
    soup = bs(req.content, 'lxml')

    for a in soup.find_all('a', href=True):
            if '/en/detail/' in a['href']:
                    full_link = urljoin(base_url, a['href'].split('?')[0])  # Remove tracking/query params
            if full_link not in link_list:
                link_list.append(full_link)
    """
    # Loop through the links and get the prices
    parameter = r'\d[\d\s,]*' #looks at numbers with spaces and commas
    for prop in link_list:
        try:
            driver.get(prop)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "detail__header_price_data"))
            )
            # Gets price if present in html
            price_elem = driver.find_element(By.CLASS_NAME, "detail__header_price_data")
            match = re.findall(parameter, price_elem.text)
            if match:
                number_str = re.sub(r'[^\d]', '', match[0])  # Deletes all except number
                price = int(number_str)
                print(f"Price : {price}")
            else:
                print("None")

        except Exception as e:
            print(f"{prop} => Error: {e}")
    """
    # Number of rooms


    # Get property state
    """try:
        productsRating = soup.find('h2', string=lambda s: "Customer reviews" in s).find_next_siblings()[0].find('strong').text
    except:
        productsRating = None
        <h4><p>Excellent</p>"""


    # Loop through the links and get the requested data
    df_data = []
    df_property = []
    # Define headers to mimic a browser (avoids blocks or bot detection)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/91.0.4472.124 Safari/537.36"
    }
    for prop in link_list:
        try:
            print(f"\n Checking: {prop}")
            detail_resp = session.get(prop, headers=headers, timeout=20)

            # HTTP status code check
            if detail_resp.status_code != 200:
                print(f"Error: Received HTTP {detail_resp.status_code}")
                continue

            detail_soup = bs(detail_resp.content, "lxml")
            
            #
            # Try finding the price element
            #
            
            # finding parent <ul> tag
            ul_tag = detail_soup.find("div", {'class':'financial w-100'}).find("ul")
                        
            # Checking ul tags
            if ul_tag:
                # find li tags
                li_tags = ul_tag.find_all("li") 
                for li in li_tags:
                    if 'Price' in li.get_text():
                        text = li.get_text(strip=True)
                        print(text)   
                        # Regex to capture numbers
                        match = re.search(r"(\d[\d\s\u202f]*\d)", text)
                        raw_price = match.group(1)  # matching the first subgroup
                        price_property = int(re.sub(r"[\s\u202f]", "", raw_price)) #replaces empty except number and cinvert to int
                        print(price_property)
            else:
                print("ul tag not found")
            
            
            # find all divs with class "data-row-wrapper"
            div_tags = detail_soup.find_all("div", {'class': 'data-row-wrapper'})

            for div_tag in div_tags:
                h_tags = div_tag.find_all("h4")
                p_tags = div_tag.find_all("p")  # details

                # Concatenate h4[i] + p[i] 
                for h, p in zip(h_tags, p_tags):
                    text1 = h.get_text(strip=True)
                    text2 = p.get_text(strip=True) 
                    combined = f"{text1}: {text2}"
                    print(combined)
                    df_data.append(combined)
            
        
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

    # looping from list, clean data 
    pattern = r"(\d[\d\s\u202f]*\d)"
    pat_only_number = r"[\s\u202f]"

    for df_el in df_data:
        if 'Number of bedrooms' in df_el:
            # Regex to capture numbers
            match = re.search(r'[0-9]+', df_el)
            numb_rooms_property = int(match.group()) #replaces empty except number and cinvert to int
            print(numb_rooms_property)
            
        if 'Livable surface' in df_el:
            # Regex to capture numbers
            match = re.search(pattern, df_el)
            raw_livable = match.group(1)  # matching the first subgroup
            livable_property = int(re.sub(pat_only_number, "", raw_livable)) #replaces empty except number and cinvert to int
            print(livable_property)
        

        if 'Kitchen equipment' in df_el:
            if 'Super equipped' in df_el:
                kitchen_equiped = 1
            else:
                kitchen_equiped = 0
        
        #if 'Furnished: Yes' in df_el:
            furnished = 1
        
        if 'Surface terrace' in df_el:
            terrace = 1
            # Regex to capture numbers
            match = re.search(pattern, df_el)
            raw_terrace = match.group(1)  # matching the first subgroup
            surface_terrace_property = int(re.sub(pat_only_number, "", raw_terrace)) #replaces empty except number and cinvert to int
            print(surface_terrace_property)
        
        if 'Surface garden' in df_el:
            garden = 1
            # Regex to capture numbers
            match = re.search(pattern, df_el)
            raw_garden = match.group(1)  # matching the first subgroup
            surface_garden_property = int(re.sub(pat_only_number, "", raw_garden)) #replaces empty except number and cinvert to int
            print(surface_garden_property)
        
        if 'Total land surface' in df_el:
            # Regex to capture numbers
            match = re.search(pattern, df_el)
            raw_land = match.group(1)  # matching the first subgroup
            surface_land_property = int(re.sub(pat_only_number, "", raw_land)) #replaces empty except number and cinvert to int
            print(surface_land_property)
        
        if 'Number of facades' in df_el:
            # Regex to capture numbers
            match = re.search(r'[0-9]+', df_el)
            facades_property = int(match.group()) #replaces empty except number and cinvert to int
            print(facades_property)
        
        #if 'Swimming pool: Yes':
        #    print(1)
            
        if 'State of the property' in df_el:
            match = re.search(r'[^.+:]+$', df_el)
            print(match.group())
time.sleep(5)  
        
        #else:
        #    print(0)
            
            




<>:122: SyntaxWarning: invalid escape sequence '\d'
<>:122: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Emmanuel\AppData\Local\Temp\ipykernel_476\2424405141.py:122: SyntaxWarning: invalid escape sequence '\d'
  """


200
678
34
200

 Checking: []
Network error: Failed to parse: []

 Checking: https://immovlan.be/en/detail/duplex/for-sale/1000/brussels/vbc89694
Price: 299 000 €
299000
State of the property: Excellent
Build Year: 2017
Availability: On contract
Number of bedrooms: 1
Surface bedroom 1: 20 m²
Livable surface: 55 m²
Furnished: No
Surface of living-room: 25 m²
Cellar: Yes
Kitchen equipment: Super equipped
Surface kitchen: 4 m²
Number of bathrooms: 1
Number of showers: 1
Number of toilets: 1
Type of heating: Gas
Type of glazing: Double glass
Entry phone: Yes
Elevator: Yes
Access for disabled: No
Floor of appartment: 3
Orientation of the front facade: South-east
Number of facades: 2
Number of floors: 4
Sewer Connection: Yes
Gas: Yes
Running water: Yes
Swimming pool: No
Specific primary energy consumption: 119 kWh/m²/year
Yearly total primary energy consumption: 6555 kWh/year
Validity date EPC/PEB: 18/08/2029
CO2 emission: 23
Certification - Electrical installation: yes, certificate in accor

In [3]:
import requests

url = "https://immovlan.be/fr/detail/maison/a-vendre/8480/ichtegem/rbt70221"
headers =  {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) '
            'Gecko/20100101 Firefox/115.0'
        ),
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }

response = requests.get(url, headers=headers)
print(response.status_code)

200


In [ ]:
"""
    Locality
    Type of property (House/apartment)
    Subtype of property (Bungalow, Chalet, Mansion, ...)
    Price
    Type of sale (Exclusion of life sales)
    Number of rooms
    Living Area
    Fully equipped kitchen (Yes/No)
    Furnished (Yes/No)
    Open fire (Yes/No) NA
    Terrace (Yes/No)
        If yes: Area
    Garden (Yes/No)
        If yes: Area
    Surface of the land
    Surface area of the plot of land
    Number of facades
    Swimming pool (Yes/No)
    State of the building (New, to be renovated, ...)
"""

In [38]:
from urllib.parse import urljoin

from bs4 import BeautifulSoup as bs

import requests
import csv
import re
import time

session = requests.Session()
base_url = 'https://immovlan.be/'

with open('./src/cities2.csv', newline='') as csvfile:

    csv_reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    for row, col in enumerate(csv_reader):
        if row == 1:
            zip_code = f'{col[0]}-{col[1]}'

page = 1

#zip_code = '1000-Brussels'
# select only houses, appartments
appartment_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=apartmen&towns={zip_code}&page={page}noindex=1'
house_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&towns={zip_code}&noindex=1'

headers =  {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) '
            'Gecko/20100101 Firefox/115.0'
        ),
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }



req = session.get(urljoin(base_url, appartment_endpoint), headers=headers)
#time.sleep(5)
print(req.status_code)
# for specific elements: driver.find_elements_by_xpath(xpath_expression)
soup = bs(req.content, 'lxml')

# Get total listed properties for search
texts = soup.find_all('div', {'class':'col-12 mb-2'})
for text in texts:
    text_listed_properties = text.get_text()

# Get numbers of listed properties
# This pattern matches the beginning of the string ^, skips any non-digit characters \D*, and then captures the first sequence of digits (\d+).
parameter = r'^\D*(\d+)'
total_listed_properties = int(re.findall(parameter, text_listed_properties)[0])

# Divide by 20 to assess number of pages
numb_of_pages = int(total_listed_properties/20)
if numb_of_pages%20 > 0:
    numb_of_pages += 1

link_list = []
full_link = []

for page in range(numb_of_pages):
    
    appartment_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=apartmen&towns={zip_code}&page={page}noindex=1'
    house_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&towns={zip_code}&noindex=1'

    req = session.get(urljoin(base_url, appartment_endpoint), headers=headers)
    print(req.status_code)
    soup = bs(req.content, 'lxml')

    for a in soup.find_all('a', href=True):
            if '/en/detail/' in a['href']:
                    full_link = urljoin(base_url, a['href'].split('?')[0])  # Remove tracking/query params
            if full_link not in link_list:
                link_list.append(full_link)

    # Loop through the links and get the requested data
    df_data = []
    df_property = [zip_code]
    # Define headers to mimic a browser (avoids blocks or bot detection)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/91.0.4472.124 Safari/537.36"
    }
    for prop in link_list:
        try:
            print(f"\n Checking: {prop}")
            detail_resp = session.get(prop, headers=headers, timeout=20)

            # HTTP status code check
            if detail_resp.status_code != 200:
                print(f"Error: Received HTTP {detail_resp.status_code}")
                continue

            detail_soup = bs(detail_resp.content, "lxml")
            
            #
            # Try finding the price element
            #
            
            # finding parent <ul> tag
            ul_tag = detail_soup.find("div", {'class':'financial w-100'}).find("ul")
                        
            # Checking ul tags
            if ul_tag:
                # find li tags
                li_tags = ul_tag.find_all("li") 
                for li in li_tags:
                    if 'Price' in li.get_text():
                        text = li.get_text(strip=True)
                        print(text)   
                        # Regex to capture numbers
                        match = re.search(r"(\d[\d\s\u202f]*\d)", text)
                        raw_price = match.group(1)  # matching the first subgroup
                        price_property = int(re.sub(r"[\s\u202f]", "", raw_price)) #replaces empty except number and cinvert to int
                        df_property.append(price_property)
            else:
                print("ul tag not found")
            #time.sleep(2)   #second waiting 
            
            # find all divs with class "data-row-wrapper"
            div_tags = detail_soup.find_all("div", {'class': 'data-row-wrapper'})

            for div_tag in div_tags:
                h_tags = div_tag.find_all("h4")
                p_tags = div_tag.find_all("p")  # details

                # Concatenate h4[i] + p[i] 
                for h, p in zip(h_tags, p_tags):
                    text1 = h.get_text(strip=True)
                    text2 = p.get_text(strip=True) 
                    combined = f"{text1}: {text2}"
                    #print(combined)
                    df_data.append(combined)
            
        
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}")
        except Exception as e:
            print(f"Unexpected error: {e}")

        # looping from list, clean data 
        pattern = r"(\d[\d\s\u202f]*\d)"
        pat_only_number = r"[\s\u202f]"

        for df_el in df_data:
            if 'Number of bedrooms' in df_el:
                # Regex to capture numbers
                match = re.search(r'[0-9]+', df_el)
                numb_rooms_property = int(match.group()) #replaces empty except number and cinvert to int
                df_property.append(numb_rooms_property)
                
            if 'Livable surface' in df_el:
                # Regex to capture numbers
                match = re.search(pattern, df_el)
                raw_livable = match.group(1)  # matching the first subgroup
                livable_property = int(re.sub(pat_only_number, "", raw_livable)) #replaces empty except number and cinvert to int
                df_property.append(livable_property)

            if 'Kitchen equipment' in df_el:
                if 'Super equipped' in df_el:
                    kitchen_equiped = 1
                else:
                    kitchen_equiped = 0
                df_property.append(kitchen_equiped)
            
            if 'Furnished: Yes' in df_el:
                furnished = 1
                df_property.append(furnished)
            
            if 'Surface terrace' in df_el:
                terrace = 1
                df_property.append(terrace)
                # Regex to capture numbers
                match = re.search(pattern, df_el)
                raw_terrace = match.group(1)  # matching the first subgroup
                surface_terrace_property = int(re.sub(pat_only_number, "", raw_terrace)) #replaces empty except number and cinvert to int
                df_property.append(surface_terrace_property)
            
            if 'Surface garden' in df_el:
                garden = 1
                # Regex to capture numbers
                match = re.search(pattern, df_el)
                raw_garden = match.group(1)  # matching the first subgroup
                surface_garden_property = int(re.sub(pat_only_number, "", raw_garden)) #replaces empty except number and cinvert to int
                df_property.append(surface_garden_property)
            
            if 'Total land surface' in df_el:
                # Regex to capture numbers
                match = re.search(pattern, df_el)
                raw_land = match.group(1)  # matching the first subgroup
                surface_land_property = int(re.sub(pat_only_number, "", raw_land)) #replaces empty except number and cinvert to int
                df_property.append(surface_land_property)
            
            if 'Number of facades' in df_el:
                # Regex to capture numbers
                match = re.search(r'[0-9]+', df_el)
                facades_property = int(match.group()) #replaces empty except number and cinvert to int
                df_property.append(facades_property)
            
            #if 'Swimming pool: Yes':
            #    print(1)
                
            if 'State of the property' in df_el:
                match = re.search(r'[^.+:]+$', df_el)
                state_property = match.group()
                df_property.append(state_property)           
        print(df_property) 

200
200

 Checking: []
Network error: Failed to parse: []
['1000-Bruxelles']

 Checking: https://immovlan.be/en/detail/duplex/for-sale/1000/brussels/vbc89694
Price: 299 000 €
['1000-Bruxelles', 299000, ' Excellent', 1, 55, 1, 2]

 Checking: https://immovlan.be/en/detail/apartment/for-sale/1000/brussels/vbc89261
Price: 1 275 000 €
['1000-Bruxelles', 299000, ' Excellent', 1, 55, 1, 2, 1275000, ' Excellent', 1, 55, 1, 2, ' Excellent', 3, 219, 1, 1, 2, 1, 13]

 Checking: https://immovlan.be/en/detail/residence/for-sale/1000/brussels/vbc87245
Price: 549 000 €
['1000-Bruxelles', 299000, ' Excellent', 1, 55, 1, 2, 1275000, ' Excellent', 1, 55, 1, 2, ' Excellent', 3, 219, 1, 1, 2, 1, 13, 549000, ' Excellent', 1, 55, 1, 2, ' Excellent', 3, 219, 1, 1, 2, 1, 13, ' To be renovated', 6, 365, 2, 1, 17, 136]

 Checking: https://immovlan.be/en/detail/apartment/for-sale/1000/brussels/vbc91627
Price: 265 000 €
['1000-Bruxelles', 299000, ' Excellent', 1, 55, 1, 2, 1275000, ' Excellent', 1, 55, 1, 2, ' Ex

KeyboardInterrupt: 

In [ ]:
headers =  {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) '
            'Gecko/20100101 Firefox/115.0'
        ),
        'Accept': (
            'text/html,application/xhtml+xml,application/xml;q=0.9,'
            'image/avif,image/webp,*/*;q=0.8'
        ),
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'TE': 'trailers',
    }
session = requests.Session()
base_url = 'https://immovlan.be/'
property_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=appartment&provinces=brussels&page=1noindex=1'
            #house_endpoint = f'en/real-estate?transactiontypes=for-sale,in-public-sale&propertytypes=house&towns={zip_code}&noindex=1'

req = session.get(urljoin(base_url, property_endpoint), headers=headers)
print(req.status_code)
soup = bs(req.content, 'lxml')

for a in soup.find_all('a', href=True):
    if '/en/detail/' in a['href']:
        full_link = urljoin(base_url, a['href'].split('?')[0])  # Remove tracking/query params
        if full_link not in link_list:
            link_list.append(full_link)

      # Get total listed properties for search
        texts = soup.find_all('div', {'class':'col-12 mb-2'})
        text_listed_properties = 0
        for text in texts:
            text_listed_properties = text.get_text()
            break
        
        # Get numbers of listed properties
        # This pattern matches the beginning of the string ^, skips any non-digit characters \D*, and then captures the first sequence of digits (\d+).
        parameter = r'^\D*(\d+)'
        try:
            total_listed_properties = int(re.findall(parameter, text_listed_properties)[0])
        except:
            print(f"Could not find number of listings for {province} - {type}")
            continue

        # Divide by 20 to assess number of pages
        numb_of_pages = int(total_listed_properties/20)
        if numb_of_pages%20 > 0:
            numb_of_pages += 1
        print(f"Total properties: {total_listed_properties} | Pages: {numb_of_pages}")        
